In [1]:
import xarray as xr
import matplotlib.pyplot as plt

In [3]:
# statistics
norm_urls = {}
norm_urls["ocn"] = {
    "mean": "gcs://noaa-ufs-gefsv13replay/ufs-hr1/0.25-degree-subsampled/06h-freq/zarr/mom6.fvstatistics.l10.1993-2019/mean_by_level.zarr",
    "std": "gcs://noaa-ufs-gefsv13replay/ufs-hr1/0.25-degree-subsampled/06h-freq/zarr/mom6.fvstatistics.l10.1993-2019/stddev_by_level.zarr",
    "stddiff": "gcs://noaa-ufs-gefsv13replay/ufs-hr1/0.25-degree-subsampled/06h-freq/zarr/mom6.fvstatistics.l10.1993-2019/diffs_stddev_by_level.zarr",
}

In [4]:
std_diff = xr.open_zarr(norm_urls["ocn"]["stddiff"], storage_options={"token":"anon"})

In [5]:
std_diff

<xarray.Dataset> Size: 500B
Dimensions:  (z_l: 10, z_i: 11)
Coordinates:
  * z_i      (z_i) float64 88B 0.0 1.031 4.498 10.8 ... 114.0 190.2 352.3 483.1
  * z_l      (z_l) float64 80B 0.5154 2.765 7.648 15.95 ... 152.1 271.2 417.7
Data variables:
    LW       float32 4B ...
    SSH      float32 4B ...
    SW       float32 4B ...
    so       (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>
    temp     (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>
    uo       (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>
    vo       (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>

In [7]:
# Using threshold on the std of diffs, the mean std would be bumped by n times.  
n = 6
std_diff_ntimes = 6*std_diff.copy()

In [8]:
std_diff_ntimes.so.values

array([0.17697182, 0.14771669, 0.10736478, 0.07246329, 0.05105908,
       0.03481008, 0.02430277, 0.01689238, 0.01051989, 0.00735667])

In [9]:
std_diff.so.values

array([0.0294953 , 0.02461945, 0.01789413, 0.01207722, 0.00850985,
       0.00580168, 0.00405046, 0.0028154 , 0.00175331, 0.00122611])

In [12]:
std_diff_ntimes.attrs["description"] = "Standard deviation of the 6 hour increments of prognostic ocean variables multiplied by 6. This multiplier is based on a threshold std below which all values are disregarded for computing mean std."

In [13]:
std_diff_ntimes

<xarray.Dataset> Size: 500B
Dimensions:  (z_i: 11, z_l: 10)
Coordinates:
  * z_i      (z_i) float64 88B 0.0 1.031 4.498 10.8 ... 114.0 190.2 352.3 483.1
  * z_l      (z_l) float64 80B 0.5154 2.765 7.648 15.95 ... 152.1 271.2 417.7
Data variables:
    LW       float32 4B 104.1
    SSH      float32 4B 0.07528
    SW       float32 4B 1.706e+03
    so       (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>
    temp     (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>
    uo       (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>
    vo       (z_l) float64 80B dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    description:  Standard deviation of the 6 hour increments of prognostic o...

In [15]:
# Append to existing zarr stores: remove capitals and specials in their equivalents in lower case
gcp_loc = "gcs://noaa-ufs-gefsv13replay/ufs-hr1/0.25-degree-subsampled/06h-freq/zarr/mom6.fvstatistics.l10.1993-2019/sixtimes/diffs_stddev_by_level.zarr"
std_diff_ntimes.to_zarr(gcp_loc, mode="w-", storage_options={"token":"/globAl/homes/n/n@garwal/.gcs/replay-service-account.json"})